In [1]:
%matplotlib inline

import scanpy as sc
import pandas as pd
import os
import seaborn as sns
import numpy as np

In [2]:
adata_file = '/home/ngr4/project/scnd/data/processed/mouse_200614.h5ad'
adata = sc.read(adata_file)

In [3]:
tdata = adata[(adata.obs['timepoint']=='18wk') & (adata.obs['ctype']=='Purkinje cell'), :]
ttdata = tdata.raw.to_adata() 
check = True
if check:
    print(ttdata[0, :].X.max()) # should be count-like
    print(tdata[0, :].X.max()) # should be normalized
del tdata

2787.0
22.485828


In [4]:
sc.pp.normalize_total(ttdata, target_sum=1e6)
if check:
    print(ttdata[0, :].X.max())

157217.8


In [5]:
goi = pd.read_csv('/home/ngr4/project/scnd/results/201105_prioritizedv2_mouse_MERFISH_gene_list_summary.csv', index_col=0)

In [10]:
goi['scRNAseq_aveCPM_18wkPC'] = np.nan
for i, g in enumerate(goi['Gene Name'].dropna().to_list()):
    try:
        np.mean(ttdata[:, g].X)
    except KeyError:
        print('Gene {} not in data'.format(g))
#         goi['scRNAseq_aveCPM_18wkPC'] = 'Not in scRNA-seq'
        continue
    goi.loc[goi['Gene Name']==g, 'scRNAseq_aveCPM_18wkPC'] = np.mean(ttdata[:, g].X)

Gene Neurod1 not in data
Gene Lphn3 not in data
Gene Ald1l1 not in data
Gene Zfp24/Zfp191 not in data


In [11]:
goi

,Gene Name,ENSEMBL ID,FPKM (mouse bulk RNA-seq),Reason for selection,ProbeNum,scRNAseq_aveCPM_18wkPC
Gene #,,,,,,
1.0,Gabra6,ENSMUST00000020703.12,275.22000,Marker gene for GC (candidate),NaN,78.883316
2.0,Slc17a7,ENSMUST00000085374.6,132.95500,Marker gene for GC (candidate),NaN,18.883928
3.0,Cbln3,ENSMUST00000063871.12,306.71700,Marker gene for GC (candidate),NaN,34.675301
4.0,Neurod1,ENSMUST00000041099.4,171.94600,Marker gene for GC (candidate),NaN,NaN
5.0,Cbln1,ENSMUST00000034076.15,283.96100,Marker gene for GC (candidate),NaN,17.649687
...,...,...,...,...,...,...
137.0,Ntng1,ENSMUST00000133268.8,7.25273,up-regulated in 30wk UBCs (EMD),NaN,73.132294
138.0,Rapgefl1,ENSMUST00000107479.2,12.20140,up-regulated in 30wk UBCs (l2fc),NaN,24.523289
139.0,Malat1,ENSMUST00000172812.2,98.45000,ncRNA,NaN,128120.234375


In [12]:
goi.to_csv('/home/ngr4/project/scnd/results/201105_prioritizedv2_mouse_MERFISH_gene_list_summary_withscRNAseq.csv')

In [8]:
for g in goi['Gene Name'].dropna().to_list():
    if not goi.loc[goi['Gene Name']==g, 'ENSEMBL ID'].isin(adata.var['gene_ids'].to_list()).any():#if goi.loc[goi['Gene Name']==g, 'ENSEMBL ID'].isin(adata.var['gene_ids'].to_list()):
        print(g)


Gabra6
Slc17a7
Cbln3
Neurod1
Cbln1
Slc17a6
Eomes
Nwd2
Lrfn5
Asic5
Atp2a3
Calb1
Car8
Ppp1r17
Fam107b
Anks1b
Gad1
Gad2
Ntn1
Megf10
Adarb2
Lphn3
Ptprk
Eml5
Grin2b
Grin3a
Nrg1
Pax2
Elavl2
Ald1l1
Rorb
Slc1a2
Atp1a2
Gdf10
Nhsl1
Slc1a3
Ank2
Mpp6
Pdgfra
Cspg4
Smoc1
Pcdh15
Tnr
Hapln2
Mag
Mog
Opalin
Cnp
Mobp
Plp1
Prr5l
Dock8
Cx3cr1
Dock2
P2ry12
Inpp5d
Flt1
Rgs5
Slco1a4
Ptprg
Dcn
Lum
Sema3g
Cped1
Slc1a6
Cacna1a
Trpc3
Rgs8
Homer3
Atp2a2
Ryr1
Ryr2
Itpr1
Grm1
Grm5
Plcb1
Plcb3
Prkca
Prkcg
Prkg1
Dagla
Grid2
Grid2ip
Gria2
Gria3
Gria4
Pcdh9
Cadm2
Cntnap2
Nlgn1
Nrxn3
Trf
Enpp6
Ascl1
Id2
Id4
Myt1
Nkx2-2
Olig1
Olig2
Tcf7l2
Sox9
Sox10
Sox17
Zfp24/Zfp191
Zfp488
Zfp536
Myrf
Rxrg
Rxra
Yy1
Gpr17
Pde10a
Epn1
Etl4
Clic6
Col18a1
Itgbl1
Snhg11
Polr2a
Hs6st3
Pak3
Ano4
Zfhx3
Kcnq2
Kcnq3
Baiap3
Ndst4
Kcnh5
Zfp804a
Emc7
Tcf4
Kcnd2
Slc15a2
Adcy2
Pde3a
Ntng1
Rapgefl1
Malat1
Neat1


In [9]:
goi.loc[goi['Gene Name']==g, 'ENSEMBL ID'].values[0]

'ENSMUST00000173672.1'